In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


# duplicate detection in exome 200k
## Yosuke Tanigawa, 2020/12/17

In the exome 200k dataset in UK Biobank, we find 163 pairs (326 variant IDs) of "duplicated" variants. Each pair represents the exact same variant, if you take `uniq` on (CHROM, POS, REF, ALT) tupple, but they are stored as different variants.


In [5]:
pvar_f <- '/oak/stanford/groups/mrivas/ukbb24983/exome/pgen/oqfe_2020/ukb24983_exomeOQFE.pvar.zst'
in_f <- file.path(dirname(pvar_f), 'ukb24983_exomeOQFE.duplicates.noID.tsv')
out_f <- file.path(dirname(pvar_f), 'ukb24983_exomeOQFE.duplicates.tsv')


In [3]:
cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}


In [4]:
fread(cmd=paste(cat_or_zcat(pvar_f), pvar_f), colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> pvar_df


In [8]:
in_f %>% fread() %>% rename('CHROM'='#CHROM') -> df


In [10]:
pvar_df %>% mutate(sort_order = 1:n()) %>% 
inner_join(df) %>%
arrange(sort_order) %>%
select(-sort_order) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


Joining, by = c("CHROM", "POS", "REF", "ALT")

